In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator


from sklearn.metrics import classification_report, confusion_matrix


Using TensorFlow backend.


In [3]:
train_data_dir = '../Data/chest_xray/chest_xray/train'
val_data_dir = '../Data/chest_xray/chest_xray/val'
test_data_dir = '../Data/chest_xray/chest_xray/test'

In [4]:
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_data_dir, batch_size=16,target_size=(224,224))

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_data_dir,batch_size=5216,target_size=(224,224))

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_data_dir, batch_size=624,target_size=(224,224))

# Creating the datasets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
train_labels_final = train_labels[:,1]
val_labels_final = val_labels[:,1]
test_labels_final = test_labels[:,1]

In [5]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(1, activation="sigmoid")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
for layer in base_model.layers:
    layer.trainable = False


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history_x = model.fit(
        train_images,
        train_labels_final,
        epochs=6,
        batch_size=50,
        validation_data=(val_images,val_labels_final),
        validation_steps=int(0.75 * 624 / 64),
        verbose =1,
        workers = -1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 58 samples, validate on 16 samples
Epoch 1/6


In [ ]:
for layer in base_model.layers:
    layer.trainable = True

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history_x_2= model.fit(
        train_images,
        train_labels_final,
        epochs=3,
        batch_size=50),
        validation_data=(val_images,val_labels_final),
        validation_steps=int(0.75 * 624 / 64),
        verbose =1,
        workers = -1,
        )
# saving weights 
model.save_weights('Xception_weights_1.h5')  
model.save('Xception_try_1.h5')

In [ ]:
print(classification_report(test_generator.classes,test_predic))